In [4]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [5]:
with open('/Users/JoshuaRohn/.secret/yelp_api.json')as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [6]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [7]:
LOCATION = 'Chicago, IL'
TERM = 'Pizza'

In [8]:
JSON_FILE = "Data/results_in_progress_Chicago_pizza.json"
JSON_FILE

'Data/results_in_progress_Chicago_pizza.json'

In [9]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Chicago_pizza.json not found. Saving empty list to file.


In [10]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
total_results = results['total']
total_results

5300

In [13]:
results_per_page = len(results['businesses'])
results_per_page

20

In [14]:
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

265

In [15]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [16]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/265 [00:00<?, ?it/s]

In [17]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/265 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [18]:
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [19]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [20]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_Chicago_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


265

In [21]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/265 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [22]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,DXwSYgiXqIVNdO9dazel6w,pequods-pizzeria-chicago,Pequod's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/8QJUNb...,False,https://www.yelp.com/biz/pequods-pizzeria-chic...,7738,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 41.92187, 'longitude': -87.664486}","[pickup, delivery]",$$,"{'address1': '2207 N Clybourn Ave', 'address2'...",+17733271512,(773) 327-1512,2158.708458
1,8vFJH_paXsMocmEO_KAa3w,lou-malnatis-pizzeria-chicago,Lou Malnati's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/abCXuj...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,7192,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.890357, 'longitude': -87.633704}","[delivery, pickup]",$$,"{'address1': '439 N Wells St', 'address2': '',...",+13128289800,(312) 828-9800,4000.999053
2,Bxd9Y2PsdvIOGIKFQh53RQ,giordanos-chicago-30,Giordano's,https://s3-media1.fl.yelpcdn.com/bphoto/OKxuR9...,False,https://www.yelp.com/biz/giordanos-chicago-30?...,3611,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.88513231411071, 'longitude': -...",[delivery],$$,"{'address1': '130 E Randolph St', 'address2': ...",+13126161200,(312) 616-1200,4994.255603
3,yMcPQm1ywLpXQX9_9Ct00g,bonci-chicago-6,Bonci,https://s3-media3.fl.yelpcdn.com/bphoto/xOqQof...,False,https://www.yelp.com/biz/bonci-chicago-6?adjus...,13,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 41.93211641571614, 'longitude': -...",[],NaN,"{'address1': '1855 W Diversey Pkwy', 'address2...",,,3004.235582
4,wiNuegM8MNw9WIP0wfMmeA,coalfire-pizza-chicago,Coalfire Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/VYA0aN...,False,https://www.yelp.com/biz/coalfire-pizza-chicag...,1200,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 41.89075, 'longitude': -87.65988}","[delivery, pickup]",$$,"{'address1': '1321 W Grand Ave', 'address2': '...",+13122262625,(312) 226-2625,2170.786489


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,NMfvUzRbNu5bUt4E_psKfg,macs-wood-grilled-chicago,Mac's Wood Grilled,https://s3-media3.fl.yelpcdn.com/bphoto/p72WsL...,False,https://www.yelp.com/biz/macs-wood-grilled-chi...,215,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,"{'latitude': 41.9031375, 'longitude': -87.6725...",[pickup],$$,"{'address1': '1801 W Division St', 'address2':...",+17737824400,(773) 782-4400,485.949174
996,0UfOragUQsYgPWy7cucWyA,capri-deli-and-pizza-palatine,Capri Deli & Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/PtcXvp...,False,https://www.yelp.com/biz/capri-deli-and-pizza-...,330,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 42.1394998, 'longitude': -88.0219...","[delivery, pickup]",$,"{'address1': '1625 N Rand Rd', 'address2': '',...",+18473599106,(847) 359-9106,38624.391093
997,9stAvpo2mu_UiVOO18ChiA,giordanos-lincolnwood,Giordano's,https://s3-media2.fl.yelpcdn.com/bphoto/KsXiS0...,False,https://www.yelp.com/biz/giordanos-lincolnwood...,43,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 41.998935, 'longitude': -87.720619}",[delivery],$$,"{'address1': '6485 N Lincoln Ave', 'address2':...",+18479830223,(847) 983-0223,11013.134900
998,RGnU238ttymbm4KQTdon2A,penguinos-pizza-buffalo-grove,Penguino's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/wPBKHd...,False,https://www.yelp.com/biz/penguinos-pizza-buffa...,81,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 42.1387951, 'longitude': -87.96195}","[delivery, pickup]",$,"{'address1': '141 W Dundee Rd', 'address2': ''...",+18474590002,(847) 459-0002,34948.653748
999,iZ_vaM2NvBghDqhHdh7D7g,home-run-inn-pizza-chicago-6,Home Run Inn Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/wWZALh...,False,https://www.yelp.com/biz/home-run-inn-pizza-ch...,224,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 41.78720430408615, 'longitude': -...",[],$$,"{'address1': '5700 S Cicero Ave', 'address2': ...",+18722407730,(872) 240-7730,14254.110390


In [23]:
final_df.duplicated(subset='id').sum()

0

In [24]:
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [25]:
final_df.to_csv('Data/final_results_Chicago_pizza.csv.gz', compression='gzip',index=False)